#### Extracting data from JSON to dataframe
extract essential information for the analysis
"title","views", "likes","comments", 'duration','tag','description', 'channel', 'category', 'published'

In [1]:
#extracting json to df
import os, json
import pandas as pd
import numpy as np
import glob
pd.set_option('display.max_columns', None)

dir = 'data'
path = os.path.join(dir, '**/*.json')
file_list = glob.glob(path)
print(file_list)
dfs = list()

for file in file_list:
    with open(file, 'r') as f:
        data = json.load(f)
    channel_id, stats = data.popitem()
    pchannel_stats = stats["channel_statistics"]
    video_stats = stats["video_data"]
    sorted_vids = sorted(video_stats.items(), key=lambda item: int(item[1]["viewCount"]),reverse=True)
    stats = []
    for vid in sorted_vids:
        video_id = vid[0]
        title = vid[1]["title"]
        views = vid[1]["viewCount"]
        likes = vid[1]["likeCount"]
        duration = vid[1]['duration']
        try:
            tags = vid[1]['tags']
        except:
            pass
        description = vid[1]['description']
        #dislikes = vid[1]["dislikeCount"]
        try:
            comments = vid[1]["commentCount"]
        except:
            pass
        channel = vid[1]['channelTitle']
        cat = os.path.dirname(file).split('\\')[1]
        stats.append([title,views, likes, comments, duration, tags, description, channel, cat])
    vid_df = pd.DataFrame(stats, columns=["title","views", "likes","comments", 'duration','tag','description', 'channel', 'category'])
    dfs.append(vid_df)
df = pd.concat(dfs, ignore_index=True)

['data\\Educational\\alltime10s.json', 'data\\Educational\\matthew_santoro.json', 'data\\Educational\\national_geographic.json', 'data\\Educational\\ted-ed.json', 'data\\Educational\\tkor.json', 'data\\Travel\\insider.json', 'data\\Travel\\mark_wiens.json']


In [2]:
df.sample(10)

,title,views,likes,comments,duration,tag,description,channel,category
2002,"What We Did with the Ink from 1,000 Pens",4653622,62853,5524,PT13M47S,"[pen ink, ink, drawing, pen, pens, fountain pe...",We asked you what we should do with our bowl o...,TKOR,Educational
631,10 Countries that No Longer Exist,867355,22630,3248,PT11M20S,"[countries of the world song, countries of the...",My Discord: http://bit.ly/SantoroLounge | My T...,Matthew Santoro,Educational
443,10 Sea Creatures More Dangerous Than Sharks,41853,1124,106,PT10M40S,"[alltime 10s, alltime10, 10s, top 10, top5, ma...",Jaws has given sharks an irreparable reputatio...,Alltime10s,Educational
3440,Khua Kling Recipe (วิธีทำคั่วกลิ้งหมู) - Delic...,91360,1979,120,PT6M4S,"[khua kling recipe, วิธีทำคั่วกลิ้งหมู, วิธีทำ...",Khua kling (คั่วกลิ้ง) is an extremely common ...,Mark Wiens,Travel
313,10 Times Science Played God,99275,2023,702,PT10M17S,"[alltime 10s, alltime10, 10s, top 10, top5, ma...",Science is always progressing at an astronomic...,Alltime10s,Educational
3034,Malaysian Street Food - GENIUS MALAY FOOD in T...,4648027,57266,5563,PT31M29S,"[Terengganu, kuala terengganu, Malay food, Mal...",Watch more Malaysian food videos: https://yout...,Mark Wiens,Travel
1678,How do hard drives work? - Kanawat Senanan,2336923,61037,1643,PT5M12S,"[Senanan Kanawat, Hard Drive, TED, TED-Ed, TED...",View full lesson: http://ed.ted.com/lessons/ho...,TED-Ed,Educational
2957,The Ultimate JERUSALEM FOOD TOUR + Attractions...,12876731,154433,19450,PT41M22S,"[Jerusalem, Jerusalem food tour, arabic food t...",Join us on the ultimate Jerusalem food tour!\n...,Mark Wiens,Travel
1234,Genius: Picasso - Trailer #2 | National Geogra...,169057,1897,108,PT1M45S,"[national geographic, nat geo, natgeo, animals...",Meet the mind behind the masterpieces. Antonio...,National Geographic,Educational
1045,Unsung Heroes of Everest | Nat Geo Live,1141488,23185,830,PT14M5S,"[Nat Geo Live, Nat Geo Live!, lecture, Nationa...",With a death rate statistically higher than so...,National Geographic,Educational
